In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import numpy as np
from numpy.linalg import norm
import clingo
import psycopg2
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *

In [2]:
canonical_dict = {"sudoku":os.path.join("projects","sudoku","canonical_sudoku.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","canonical_seeknumbers2.lp"),
                 "minotaur":os.path.join("projects","minotaur","canonical_minotaur1.lp"),
                 "creek":os.path.join("projects","creek","canonical_creek.lp"),                 
                 "yosenabe":os.path.join("projects","yosenabe","canonical_yosenabe.lp"),
                 "hop":os.path.join("projects","hop","canonical_hop.lp"),
                 "lights":os.path.join("projects","lights","canonical_lights.lp")}

In [3]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp"),
                 "yosenabe":os.path.join("projects","yosenabe","instances","instance01.lp"),
                 "hop":os.path.join("projects","hop","instances","level1.lp"),
                 "lights":os.path.join("projects","lights","instances","test01.lp")}

In [4]:
solution_dict ={"sudoku":os.path.join("projects","sudoku","solutions","ex01.json"),
                 "seeknumbers":os.path.join("projects","seeknumbers","solutions","ex01.json"),
                 "minotaur":os.path.join("projects","minotaur","solutions","level01.json"),
                 "creek":os.path.join("projects","creek","solutions","ex01.json"),
                 "yosenabe":os.path.join("projects","yosenabe","solutions","solution01.txt"),
                 "hop":os.path.join("projects","hop","solutions","solution01.txt"),
                 "lights":os.path.join("projects","lights","solutions","solution01.txt")}

In [39]:
def asp_try_5(asp_file,instance):
    models = []  
    errors = []
    symbols = []
    messages = []

    def custom_logger(code, message):
        print(code)
        print(message)
        messages.append(message)
        
    def on_model(model):
        #print("Model:", model)
        models.append(model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            #for s in symbols_list:
            symbols.append(symbols_list)
        os.remove("output.txt")

    time_init = time.time() 
    asp_program = []
    # TODO: timeout of Unix systems. Wrapper around. Kill the process. 
    control = clingo.Control(['--warn=none','-t','5','-n','10'], logger=custom_logger) #, '--warn=none', '--opt-mode=optN' --time-limit=5, -t 5
    input_files = [asp_file, instance]
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
            
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    try:
        control.ground([("base", [])])
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    time_final = time.time() 
    delta_time = time_final-time_init
    
    if delta_time > 5 and len(symbols) == 0:
        errors.append("Timeout")

    #print(f"Results: {models}, {errors}, {symbols}. Timedelta {delta_time}")
    return models, errors, symbols, messages

In [40]:
def asp_try_3(asp_file,instance):
    errors = []
    messages = []
    error_dict = {}
    message_dict = {}
    def custom_logger(code, message):
        print(code)
        print(message)
        errors.append(code)
        messages.append(message)
    control = clingo.Control(logger=custom_logger,message_limit=100)
    input_files = [asp_file, instance]
    asp_program = []
    models = []
    symbols = []
    
    def on_model(model):
        print("Model:", model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            for s in symbols_list:
                symbols.append(s)
        os.remove("output.txt") 
        #print(type(model))
        #symbols = model.symbols(atoms=False,terms=True, shown=True)
        #for s in symbols:
        #    print(s.arguments)
        #    print(type(s))
        #    symbols_list.append(s)
        #print("Symbols:", symbols)
        
         #def symbols(self, atoms: bool = False, terms: bool = False, 
         #            shown: bool = False, theory: bool = False, complement: bool = False) ‑> Sequence[Symbol] 
        models.append(model)
    
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    try:
        control.add("base", [], "".join(asp_program))
    except:
        models, error_dict, message_dict, symbols
    try:
        control.ground([("base", [])])
    except:
        print("Grounding stopped because of errors")
        error_codes = ["AtomUndefined","FileIncluded","GlobalVariable","OperationUndefined","Other","RuntimeError","VariableUnbounded"]
        sub_codes = ["unsafe"]
        
        for error_code in error_codes:
            error_dict[error_code] = 0
        
        for sub_code in sub_codes:
            sub_dict[sub_code] = 0
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    sub_dict[sc] += 1
        return models, error_dict, message_dict
        
    control.configuration.solve.models = 0  # Limit the number of models to 1
    control.solve(on_model=on_model)
    
    return models, error_dict, message_dict, symbols

In [41]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [42]:
cursor = conn.cursor()
cursor.execute("SELECT file, n_errors, errors FROM panel WHERE n_errors > 0 LIMIT 1")
results = cursor.fetchall()

In [43]:
results

[('creek_from_minotaur_and_sudoku_v2_20231129.lp', 1, 'parsing failed')]

In [44]:
bad_file = results[0][0]
bad_path = os.path.join('generated_solutions',bad_file)
instance = instances_dict[bad_file.split("_")[0]]

In [45]:
models, errors, symbols, messages = asp_try_5(bad_path, instance)

MessageCode.RuntimeError
<block>:33:108-109: error: syntax error, unexpected =, expecting )



In [46]:
messages

['<block>:33:108-109: error: syntax error, unexpected =, expecting )\n']

In [47]:
symbols

[]

In [25]:
error_dict

{'AtomUndefined': 0,
 'FileIncluded': 0,
 'GlobalVariable': 0,
 'OperationUndefined': 0,
 'Other': 0,
 'RuntimeError': 4,
 'VariableUnbounded': 0}

In [26]:
messages

{'unsafe': 4}

In [ ]:
try:    
    print("No errors occurred during solving.")
except Exception as e:
    print(e)
    #if control.statistics['solving']['solvers']['conflicts'] > 0:
    print("Errors occurred during solving.")
    # Analyze the conflicts to get more information
    conflicts_analyzed = control.statistics['solving'] #['conflicts_analyzed']
    print("Conflicts analyzed: {}".format(conflicts_analyzed))

In [8]:
aux = '''<block>:2:1-91: error: unsafe variables in:
  move(X,Y,XX,YY):-[#inc_base];#Arith0=1;player(X,Y);#Arith0=(|(X-XX)|+|(Y-YY)|);not minotaur(XX,YY);not wall(XX,YY).
<block>:2:10-12: note: 'XX' is unsafe
<block>:2:13-15: note: 'YY' is unsafe'''

In [9]:
if "unsafe variables" in aux:
    print("Éxito")

Éxito


## Others

In [3]:
import pandas as pd
import psycopg2

In [4]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [5]:
def load_from_database(conn, story, version):
    cursor = conn.cursor()
    command = "SELECT * FROM experiments e JOIN results r ON e.file = r.file WHERE story = '{}' AND version = {}".format(story, version)
    print(command)
    results = None
    try:
        cursor.execute(command)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Bad Query.")
        conn.commit()
        cursor.close()
        pass
    return results

In [8]:
results = load_from_database(conn, "sudoku", 2)

SELECT * FROM experiments e JOIN results r ON e.file = r.file WHERE story = 'sudoku' AND version = 2


In [12]:
type(results[0])

tuple

In [30]:
def load_one_from_db(conn, story, version):
    cursor = conn.cursor()
    command = "SELECT * FROM experiments e JOIN results r ON e.file = r.file WHERE story = '{}' AND version = {}".format(story, version)
    #st.write(command)
    print(command)
    result_list = []
    try:
        cursor.execute(command)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Bad Query.")
        conn.commit()
        cursor.close()
        pass
    #print(results)
    print("=================")
    for r in results:
        print(r[0])
        break
        #dicc_result = {"id":r[0], "prompt1":r[1], "prompt2":r[2], "encoding":r[3], "story":r[4], "story_1":r[5], "story_2":r[6], "version":r[7], "file":r[8], "created_at":r[9], "llm":r[10], "id_result":r[11], "file2":r[12], "kpi":r[13], "diff_lines":r[14], "errors":r[15], "n_models":r[16], "symbols":r[17], "matchs":r[18], "match_ratio":r[19], "created_at_result":r[20]}
        #results.append(dicc_result)
    #st.write(result_list)
    return result_list

In [31]:
results = load_one_from_db(conn, "sudoku", 2)

SELECT * FROM experiments e JOIN results r ON e.file = r.file WHERE story = 'sudoku' AND version = 2
64


In [17]:
print(results)

[(64, 'Given a problem and some predicates, write ASP (Answer Set Programming) rules to generate the search space of possible relations. Do not repeat rules.\n            \n        Problem 1:\n                    The task of this project is to solve the Japanese grid puzzle Yosenabe using ASP. Given a grid, the task is to move each number surrounded by a frame into one of the gray areas along a straight line, respecting the following conditions:\n1. The ways of any two moved numbers must not cross or meet at any grid cell.\n2. Each gray area must be populated with at least one moved number.\n3. An area may be associated with a (positive) goal number, shown within it. If so, the numbers moved into the area must sum up exactly to the goal.\nWhile a number can be moved through an area, you may assume that a move stops at the fi\x0crst cell w.r.t. its direction of the area into which it leads.\n\n        Input_predicates 1:\n                    % There is a cell with coordinates (X,Y)\n   